In [1]:
import pandas as pd
import random
import io
import ast
import numpy as np
import os
import re
import string
import pickle
import copy
import seaborn as sns
import torch.nn.functional as functional
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tnrange
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from bs4 import BeautifulSoup
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

import torch, torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import nltk
from nltk.stem import WordNetLemmatizer
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import pairwise_distances

from time import time
from tqdm import tnrange

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Loading texts, split on train, val and test


In [2]:
!wget https://www.dropbox.com/s/n8nfxxh5azocrrm/train_LASER.csv?dl=0
!wget https://www.dropbox.com/s/je9am5c77ytfcaf/test_LASER.csv?dl=0 

--2020-04-28 14:53:30--  https://www.dropbox.com/s/n8nfxxh5azocrrm/train_LASER.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/n8nfxxh5azocrrm/train_LASER.csv [following]
--2020-04-28 14:53:30--  https://www.dropbox.com/s/raw/n8nfxxh5azocrrm/train_LASER.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca34bcbb44e474dd382eed5dbfc.dl.dropboxusercontent.com/cd/0/inline/A2uisowP5TApulpwUGS6RfD-N3vjfHW3dbYZ41BMFEotHLxtO_EqP59QK5GeDYkHTfqGOpXGF_k-Mk6h-uL5Pmz-FsxZEzA26whcbDpAG0lXzirT9F_MTzwJ1IRvCiah6rw/file# [following]
--2020-04-28 14:53:31--  https://uca34bcbb44e474dd382eed5dbfc.dl.dropboxusercontent.com/cd/0/inline/A2uisowP5TApulpwUGS6RfD-N3vjfHW3dbYZ41BMFEotHLxtO_EqP59QK5GeDYkHTfqGOpXGF_k-Mk6h-uL5Pmz-FsxZEzA26whc

In [3]:
#открываем файл с train dataset

df_train = pd.read_csv("train_LASER.csv?dl=0").dropna()

train_texts = list(df_train['abstracts'])
train_labels = list(df_train['labels'])
len(train_labels)


80948

In [4]:
#открываем файл с test_dataset

df_test = pd.read_csv("test_LASER.csv?dl=0").dropna()

test_texts = list(df_test['lemm_abstracts'])
test_labels = list(df_test['labels'])
len(test_labels)

10150

In [0]:
#разбиваем на train и val английские тексты
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, \
                                                          test_size=0.15, shuffle = True, random_state=42, stratify=train_labels)

In [6]:
print(len(train_labels))
print(len(val_labels))

68805
12143


In [0]:
test_texts[1000]

'использование биотехнология, медицинская микробиология и может быть использовано в производстве эубиотических препаратов и бифидосодержащих продуктов питания. сущность изобретения питательная среда в качестве основы содержит молочную сыворотку, осветленную последовательной обработкой обезжиренного молока кислотой и щелочью, и дрожжевой автолизат сухой или жидкий и дополнительно натрий фосфорнокислый однозамещенный двуводный, аммоний хлористый и магний сернокислый семиводный, при следующем соотношении компонентов мас. молочная сыворотка осветленная , , , лактоза , , , дрожжевой автолизат сухой или жидкий , , и , , , d,l цистеин , , , nah po • h o , , , nh cl , , , mgso • h o , , , агар агар , , , вода остальное, ph , , .'

In [7]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_labels)
#add_labels = label_encoder.fit_transform(df_add['us_code'])
y_train[:10]

array([ 58, 199, 218,   7, 196, 115,  69,   1,  13,  23])

In [8]:
y_val = label_encoder.transform(val_labels)
y_val[:10]

array([ 23,  82,  57, 164,  99, 146,  24, 161, 194,  16])

In [0]:
y_test = label_encoder.transform(test_labels)

In [10]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print(f'Using device: {DEVICE}')

Using device: cuda


### Cleaning

In [0]:
def cleaning(text):
    clean_txt = []
    for sentence in text.split('.'): 
        sentence = re.sub(r'[•⋅−,]', ' ', sentence)
        sent = re.sub(r'( )+', ' ', sentence)
     #   if len(sent.split()) > 2 and sent != 'ф лы, ил':
        if len(sent.split()) > 2 and 'ф лы' not in sent:
             clean_txt.append(sent.strip())
    return ' '.join(clean_txt) 

In [12]:
test_texts = [cleaning(text) for text in tqdm(test_texts)]
val_texts = [cleaning(text) for text in tqdm(val_texts)]
train_texts = [cleaning(text) for text in tqdm(train_texts)]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
test_texts[1000]

'использование биотехнология медицинская микробиология и может быть использовано в производстве эубиотических препаратов и бифидосодержащих продуктов питания сущность изобретения питательная среда в качестве основы содержит молочную сыворотку осветленную последовательной обработкой обезжиренного молока кислотой и щелочью и дрожжевой автолизат сухой или жидкий и дополнительно натрий фосфорнокислый однозамещенный двуводный аммоний хлористый и магний сернокислый семиводный при следующем соотношении компонентов мас молочная сыворотка осветленная лактоза дрожжевой автолизат сухой или жидкий и d l цистеин nah po h o nh cl mgso h o агар агар вода остальное ph'

In [13]:
def compute_lens(texts):
    lens = [len(text.split()) for text in texts]
    return lens

train_lens = compute_lens(train_texts)
val_lens = compute_lens(val_texts)
test_lens = compute_lens(test_texts)
train_lens[:10]

[142, 194, 152, 77, 98, 100, 149, 206, 96, 158]

### Loading MUSE embeddings

In [0]:
def load_vec(emb_path, nmax=200000):   #200000 - это максимальное число слов, которое есть в MUSE embeddings
    vectors = []
    word2id = {}
    with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        next(f)
        for i, line in enumerate(f):
            word, vect = line.rstrip().split(' ', 1)
            vect = np.fromstring(vect, sep=' ')
            assert word not in word2id, 'word found twice'
            vectors.append(vect)
            word2id[word] = len(word2id)
            if len(word2id) == nmax:
                break
    id2word = {v: k for k, v in word2id.items()}
    embeddings = np.vstack(vectors)
    return embeddings, id2word, word2id

In [15]:
#download MUSE russian embeddings
!wget https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.ru.vec

#download MUSE english embeddings
!wget https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.en.vec

--2020-04-28 14:54:37--  https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.ru.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 2606:4700:10::6816:4a8e, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 630591038 (601M) [text/plain]
Saving to: ‘wiki.multi.ru.vec’

wiki.multi.ru.vec   100%[===================>] 601.38M  33.1MB/s    in 19s     

2020-04-28 14:54:57 (32.1 MB/s) - ‘wiki.multi.ru.vec’ saved [630591038/630591038]

--2020-04-28 14:54:58--  https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.en.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 2606:4700:10::6816:4a8e, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 628614720 (599M) [text/plain]
Saving to: ‘wiki.multi.e

In [0]:
src_embeddings, src_id2word, src_word2id = load_vec('wiki.multi.en.vec')
tgt_embeddings, tgt_id2word, tgt_word2id = load_vec('wiki.multi.ru.vec')

### Texts vectorization

In [0]:
# from https://github.com/aambekar234/Cross-lingual-text-classification-using-Muse-Deeplearning/blob/master/cross-lingual-text-classification-average-vector.ipynb
#convert tokenized docs to vector embeddings by averaging
def docs_to_vector(docs, emb_type, emb_id):
    vectors = []
    cnt_found = np.zeros((len(docs)))

    for i, doc in tqdm(enumerate(docs)):
        vector = np.zeros((300,), dtype=np.float64)
        for token in doc.split():
            if token in emb_id.keys():
                vector += emb_type[emb_id[token]]
                cnt_found[i] += 1
            else:
                continue 
        vector /= cnt_found[i]                                             
        vectors.append(vector)
    return np.array(vectors), cnt_found

In [18]:
X_train, cnt_found_tr = docs_to_vector(train_texts, src_embeddings, src_word2id)
X_train.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


(68805, 300)

In [19]:
X_val, cnt_found_val = docs_to_vector(val_texts, src_embeddings, src_word2id)
X_val.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


(12143, 300)

In [20]:
X_test, cnt_found_test = docs_to_vector(test_texts, tgt_embeddings, tgt_word2id)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [21]:
#посчитаем, сколько процентов слов из документа нашлось в словаре MUSE embeddings (для каждого посчитаем)
percent_found_train = (cnt_found_tr/train_lens)*100
percent_found_val = (cnt_found_val/val_lens)*100
percent_found_test = (cnt_found_test/test_lens)*100

print('Процент найденных слов в документе, mean +- variance: \n')
print('Train: ' + str(np.round(np.mean(percent_found_train), 2)) + '+-' + str(np.round(np.std(percent_found_train),2)))
print('Validation: ' + str(np.round(np.mean(percent_found_val), 2)) + '+-' + str(np.round(np.std(percent_found_val),2)))
print('Test: ' + str(np.round(np.mean(percent_found_test), 2)) + '+-' + str(np.round(np.std(percent_found_test),2)))

Процент найденных слов в документе, mean +- variance: 

Train: 98.77+-2.25
Validation: 98.78+-2.23
Test: 88.62+-5.63


### Experiments

#### SVC Classifier

In [0]:
def predict(y_true, y_pred):
    print('micro: ', f1_score(y_true, y_pred, average='micro')*100)
    print('weighted: ', f1_score(y_true, y_pred, average='weighted')*100)

In [0]:
from sklearn import linear_model
alphas = [0.01, 0.001, 0.0001, 0.00001]
for value in alphas:
    print('alpha=', value)
    clf = linear_model.SGDClassifier(max_iter=10000, tol=1e-3, early_stopping=True, random_state=42, verbose=0,\
                                     learning_rate='adaptive', eta0 = 1e-2, n_jobs=-1, alpha=value)
    t0 = time()
    clf.fit(X_train, y_train)
    print('f1_train:' + '\n')
    predict(y_train, clf.predict(X_train))
    print('f1_val:' + '\n')
    predict(y_val, clf.predict(X_val))
    print('train and predict in ' + str(round(time() - t0, 2)) + ' seconds ')
    print('f1_test:' + '\n')
    predict(y_test, clf.predict(X_test))
    print('==========================')

alpha= 0.01
f1_train:

micro:  49.580699077101954
weighted:  48.80532499129247
f1_val:

micro:  45.3924071481512
weighted:  44.40970760990701
train and predict in 400.45 seconds 
f1_test:

micro:  13.783251231527094
weighted:  13.64545759845395
alpha= 0.001
f1_train:

micro:  48.92522345759756
weighted:  48.81138868917224
f1_val:

micro:  44.593593016552745
weighted:  44.294713954831685
train and predict in 396.63 seconds 
f1_test:

micro:  13.123152709359607
weighted:  13.09558246555811
alpha= 0.0001
f1_train:

micro:  48.55461085676913
weighted:  47.445761705750236
f1_val:

micro:  45.013588075434406
weighted:  43.65129049686091
train and predict in 399.68 seconds 
f1_test:

micro:  14.748768472906404
weighted:  14.818394705834912
alpha= 1e-05
f1_train:

micro:  48.38747184070925
weighted:  47.141938472139884
f1_val:

micro:  44.7335913695133
weighted:  43.17952137073136
train and predict in 397.37 seconds 
f1_test:

micro:  14.69950738916256
weighted:  14.83084846200956


In [0]:
y_pred = clf.predict(X_train)
y_pred[:10]

array([181, 199, 202, 116, 196, 116, 202,   1,  13, 202])

In [0]:
y_pred = clf.predict(X_val)
y_pred[:10]

array([199,   4,   0, 163,  97, 120,  30,   5,  32,  10])

In [0]:
f1_score(y_val, y_pred, average='micro')*100

12.435147821790332

####  Logistic regression and FFNN

In [0]:
random.seed(13)
np.random.seed(13)
torch.manual_seed(13)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ['PYTHONHASHSEED'] = str(13)

In [0]:
class LogisticRegression(nn.Module):
    def __init__(self, emb_dim, num_labels):
        super().__init__()
        self.linear = nn.Linear(emb_dim, num_labels)
        nn.init.xavier_normal_(self.linear.weight)

    def forward(self, emb):
        return F.log_softmax(self.linear(emb), dim=1)

In [0]:
N_EPOCHS = 600
LEARNING_RATE = 1e-2
model = LogisticRegression(300,225).to(DEVICE) 
#model = FFNN(300).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_function = nn.NLLLoss()

In [0]:
def iterate_minibatches(data, labels, batch_size=100, shuffle=True, verbose=True):
    indices = np.arange(len(data))
    if shuffle:
        indices = np.random.permutation(indices)        
    irange = tnrange if verbose else range
 #   print(indices)
    for start in irange(0, len(indices), batch_size):
        yield data[indices[start: start + batch_size]], labels[indices[start: start + batch_size]]

In [0]:
def train_model_epoch(train_dataset, labels, model, optimizer, batch_size):
    model.train()
    train_loss = 0
    train_predictions = np.array([])
    pred_labels = np.array([])
    
    for texts, labs in iterate_minibatches(np.array(train_dataset), np.array(labels),\
                                           batch_size=batch_size, shuffle=True, verbose=False):
        emb_ix = torch.tensor(texts, dtype=torch.float32).to(DEVICE) #cuda
     #   print(labels[i: i+batch_size])
        res_labels = torch.tensor(labs).to(DEVICE)     #cuda 
      #  print(labels.shape)
        optimizer.zero_grad()
        output = model(emb_ix)
 
        loss = loss_function(output, res_labels)
        train_loss += loss.item()

        y_pred = output.argmax(1).cpu().numpy()
        train_predictions = np.hstack((train_predictions, y_pred))
        pred_labels = np.hstack((pred_labels, res_labels.cpu().numpy()))

        loss.backward()
        optimizer.step()

    train_f1_micro = f1_score(pred_labels, train_predictions, average='micro')*100
    train_f1_weighted = f1_score(pred_labels, train_predictions, average='weighted')*100

    return train_loss / (len(train_dataset)//batch_size), train_f1_micro, train_f1_weighted

In [0]:
def test(test_dataset, labels, model, batch_size):
    model.eval()
#    acc = 0
    test_predictions = np.array([])
    test_loss = 0
    for texts, labs in iterate_minibatches(np.array(test_dataset), np.array(labels), \
                                           batch_size=batch_size, shuffle=False, verbose=False):
        with torch.no_grad():
            emb_ix = torch.tensor(texts, dtype = torch.float32).to(DEVICE)
            res_labels = torch.tensor(labs).to(DEVICE)  
        #  print(labels.shape)
            output = model(emb_ix)
            y_pred = output.argmax(1).cpu().numpy()
            loss = loss_function(output, res_labels)
            test_loss += loss.item()
            test_predictions = np.hstack((test_predictions, y_pred))

    test_f1_micro = f1_score(labels, test_predictions, average='micro')*100
    test_f1_weighted = f1_score(labels, test_predictions, average='weighted')*100


    return test_loss / (len(test_dataset)//batch_size), test_f1_micro, test_f1_weighted

In [0]:
X_val[0].shape

(1, 12453)

In [0]:
y_val

array([23, 82, 57, ...,  4, 48, 92])

In [61]:
print('Training...')
#best_loss = 10
max_step = 2
step=0
best_f1_micro = 1
train_time = []
for epoch in tnrange(N_EPOCHS+1):
    t0 = time()
    train_loss, train_f1_micro, train_f1_weighted = train_model_epoch(X_train, y_train, model, optimizer, batch_size=100)
    train_time.append(round(time() - t0, 2))
 #   train_loss, train_acc = train_model_epoch_sample(train_texts, labels, model, optimizer)
    test_loss, test_f1_micro, test_f1_weighted = test(X_val, y_val, model, batch_size=100)
    if test_f1_micro < best_f1_micro + 0.1:
      step += 1
      
      if step > max_step:
        break
    else:
      best_f1_micro = test_f1_micro
      step = 0
      weights = copy.deepcopy(model.state_dict())
           
    
      new_model = LogisticRegression(300,225).to(DEVICE)
      new_model.load_state_dict(weights)

    if epoch % 10 == 0:
     #   test_loss, test_f1_micro, test_f1_weighted = test(X_val, y_val, model, batch_size=300)
        print(f'\nTrain loss: {train_loss:.8f} | Train f1 micro: {train_f1_micro:.4f}  | Train f1 weighted: {train_f1_weighted:.4f}' )   
    #    print(f'Test accuracy {test_acc:.4f}')
        print(f'\nVal f1 micro: {test_f1_micro:.4f} | Val f1 weighted: {test_f1_weighted:.4f}')
print('Finished training.')

Training...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  import sys



Train loss: 4.74065873 | Train f1 micro: 11.5370  | Train f1 weighted: 8.5737

Val f1 micro: 19.6245 | Val f1 weighted: 14.2315

Train loss: 2.63296629 | Train f1 micro: 41.5726  | Train f1 weighted: 39.6138

Val f1 micro: 39.5701 | Val f1 weighted: 37.4108

Train loss: 2.31736956 | Train f1 micro: 46.3600  | Train f1 weighted: 45.2110

Val f1 micro: 43.4407 | Val f1 weighted: 42.0328

Train loss: 2.17280227 | Train f1 micro: 48.9107  | Train f1 weighted: 48.0907

Val f1 micro: 44.9889 | Val f1 weighted: 43.7385
Finished training.


In [0]:
new_model.eval()
emb_ix = torch.tensor(X_test, dtype = torch.float32).to(DEVICE)
output = new_model(emb_ix)
y_pred = output.argmax(1).cpu().numpy()

In [0]:
def compute_f1(model, data, labels):
    model.eval()
    emb_ix = torch.tensor(data, dtype = torch.float32).to(DEVICE)
    output = new_model(emb_ix)
    y_pred = output.argmax(1).cpu().numpy()
    print('f1_micro= ', np.round(f1_score(labels, y_pred, average='micro')*100, 2))
    print('f1_weighted= ', np.round(f1_score(labels, y_pred, average='weighted')*100, 2))

In [64]:
print('Train f1')
compute_f1(new_model, X_train, y_train)
print('Val f1')
compute_f1(new_model, X_val, y_val)
print('Test f1')
compute_f1(new_model, X_test, y_test)

Train f1
f1_micro=  49.12
f1_weighted=  48.24
Val f1
f1_micro=  44.92
f1_weighted=  43.64
Test f1
f1_micro=  19.7
f1_weighted=  18.06


In [0]:
print(classification_report(y_test, y_pred, digits=4))

In [65]:
print('Общее время тренировки, в секундах: ', np.sum(train_time))
print('Одна эпоха в среднем, в секундах: ', np.mean(train_time))

Общее время тренировки, в секундах:  40.940000000000005
Одна эпоха в среднем, в секундах:  1.2406060606060607


In [0]:
class FFNN(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.linear1 = nn.Linear(emb_dim, 256)
        self.linear2 = nn.Linear(256, 225)
        self.relu = nn.ReLU()
    #    self.norm1 = nn.BatchNorm1d(256)
    #    self.dropout = nn.Dropout(0.1)
        nn.init.xavier_normal_(self.linear1.weight)
        nn.init.xavier_normal_(self.linear2.weight)

    def forward(self, emb):
        output = self.linear2(self.relu(self.linear1(emb)))
        return F.log_softmax(output, dim=1)
      #  return output

In [0]:
N_EPOCHS = 300
LEARNING_RATE = 3e-4

model = FFNN(300).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_function = nn.NLLLoss()

In [68]:
print('Training...')
#best_loss = 10
max_step = 2
step=0
best_f1_micro = 1
train_time = []
for epoch in tnrange(N_EPOCHS+1):
    t0 = time()
    train_loss, train_f1_micro, train_f1_weighted = train_model_epoch(X_train, y_train, model, optimizer, batch_size=100)
    train_time.append(round(time() - t0, 2))
 #   train_loss, train_acc = train_model_epoch_sample(train_texts, labels, model, optimizer)
    test_loss, test_f1_micro, test_f1_weighted = test(X_val, y_val, model, batch_size=100)
    if test_f1_micro < best_f1_micro + 0.1:
      step += 1
      
      if step > max_step:
        break
    else:
      best_f1_micro = test_f1_micro
      step = 0
      weights = copy.deepcopy(model.state_dict())
           
    
      new_model = FFNN(300).to(DEVICE)
      new_model.load_state_dict(weights)

    if epoch % 10 == 0:
     #   test_loss, test_f1_micro, test_f1_weighted = test(X_val, y_val, model, batch_size=300)
        print(f'\nTrain loss: {train_loss:.8f} | Train f1 micro: {train_f1_micro:.4f}  | Train f1 weighted: {train_f1_weighted:.4f}' )   
    #    print(f'Test accuracy {test_acc:.4f}')
        print(f'\nVal f1 micro: {test_f1_micro:.4f} | Val f1 weighted: {test_f1_weighted:.4f}')
print('Finished training.')

Training...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  import sys



Train loss: 5.23205950 | Train f1 micro: 2.8414  | Train f1 weighted: 1.4427

Val f1 micro: 5.4929 | Val f1 weighted: 1.7173

Train loss: 3.40341167 | Train f1 micro: 22.5521  | Train f1 weighted: 18.6969

Val f1 micro: 21.9962 | Val f1 weighted: 18.1952

Train loss: 2.98561705 | Train f1 micro: 30.1940  | Train f1 weighted: 27.1134

Val f1 micro: 29.2926 | Val f1 weighted: 26.1201

Train loss: 2.75427475 | Train f1 micro: 34.7824  | Train f1 weighted: 32.2148

Val f1 micro: 33.8137 | Val f1 weighted: 31.1444

Train loss: 2.59885160 | Train f1 micro: 37.7618  | Train f1 weighted: 35.5551

Val f1 micro: 36.7537 | Val f1 weighted: 34.4440

Train loss: 2.48718861 | Train f1 micro: 40.1497  | Train f1 weighted: 38.2187

Val f1 micro: 38.7713 | Val f1 weighted: 36.7282

Train loss: 2.40485286 | Train f1 micro: 41.8879  | Train f1 weighted: 40.1807

Val f1 micro: 40.2042 | Val f1 weighted: 38.3386
Finished training.


In [69]:
print('Train f1')
compute_f1(new_model, X_train, y_train)
print('Val f1')
compute_f1(new_model, X_val, y_val)
print('Test f1')
compute_f1(new_model, X_test, y_test)

Train f1
f1_micro=  41.72
f1_weighted=  39.94
Val f1
f1_micro=  40.43
f1_weighted=  38.54
Test f1
f1_micro=  19.67
f1_weighted=  17.55


In [70]:
print('Общее время тренировки, в секундах: ', np.sum(train_time))
print('Одна эпоха в среднем, в секундах: ', np.mean(train_time))

Общее время тренировки, в секундах:  104.32
Одна эпоха в среднем, в секундах:  1.6825806451612901


#### generate embedings for lstm and cnn

In [0]:
random.seed(13)
np.random.seed(13)
torch.manual_seed(13)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ['PYTHONHASHSEED'] = str(13)

In [0]:
random_vec = np.random.normal(0, 1, size=(300, ))
random_vec /= sum(x*x for x in random_vec) ** .5

In [0]:
def generate_embeddings(docs, emb_type, emb_id, maxLen):
    vectors = []
    lengths = [len(text.split()) for text in docs]
    maxLen = min(max(lengths), maxLen or float('inf'))
 #   print(maxLen)
    for doc in docs:
        vector = []
        for token in doc.split():
            if token in emb_id.keys():
                vector.append(emb_type[emb_id[token]])
            else:
                vector.append(random_vec)
        #padd sequence for max length
        pad = maxLen - len(vector)
        if pad > 0:
            padv = np.zeros((300,),dtype=np.float64)
            for i in range(pad):
                vector.append(padv)
          
        #adjust the vector if it greater than max length
        if pad < 0:
            vector = vector[:pad]
        vectors.append(vector)
      
        
    return np.array(vectors), lengths

#### LSTM clasiifier

In [0]:
class LSTMClassifier(nn.Module):
    def __init__(self, output_size=225, hidden_size=256, layers=1, dropout = 0.0):
        super(LSTMClassifier, self).__init__()

        """
        Arguments
        ---------
        batch_lstm : Size of the batch 
        output_size : the number of classes
        hidden_size : Size of the hidden_state of the LSTM

        """
        
        self.output_size = output_size
        self.hidden_size = hidden_size

        self.layers = layers
        self.lstm = nn.LSTM(300, hidden_size, num_layers=self.layers, bidirectional=True, dropout=0.1)         #True
        self.label = nn.Linear(2*hidden_size, output_size)            #2* ... for BiLSTM
        self.dropout = dropout

    def attention_net(self, lstm_output, final_state):      
        
        hidden = final_state.squeeze(0)
        attn_weights = torch.bmm(lstm_output, hidden.unsqueeze(2)).squeeze(2)
        soft_attn_weights = F.softmax(attn_weights, 1)

        new_hidden_state = torch.bmm(lstm_output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2)

        return new_hidden_state


    def forward(self, x, lengths, batch_lstm=None):

        x = x.permute(1, 0 ,2)
        
        batch_lstm = x.size(1)   
        packed = torch.nn.utils.rnn.pack_padded_sequence(x, lengths, enforce_sorted=False)

        h_0 = Variable(torch.zeros(2*self.layers, batch_lstm, self.hidden_size)).to(DEVICE)   #.cuda()          #2 * ...
        c_0 = Variable(torch.zeros(2*self.layers, batch_lstm, self.hidden_size)).to(DEVICE)              #.cuda()
    #   print(packed)
    
      #  output, final_hidden_state = self.lstm(packed, h_0)
        output, (final_hidden_state, final_cell_state) = self.lstm(packed, (h_0, c_0))        
        final = torch.cat([final_hidden_state[0], final_hidden_state[1]], axis=1)
      #  print(final_hidden_state.shape)
    #    print(final.shape)
        unpacked_output = pad_packed_sequence(output, batch_first=True)[0]
     #   print(unpacked_output.shape)
        attn_output = self.attention_net(unpacked_output, final)                
        
        final_output = nn.Dropout(self.dropout)(self.label(attn_output)) # final_hidden_state.size() = (1, batch_size, hidden_size) & final_output.size() = (batch_size, output_size)
   #     final_output = self.label(final)
  
        return F.log_softmax(final_output, dim=1)

In [0]:
N_EPOCHS = 10
LEARNING_RATE = 1e-3

model = LSTMClassifier().to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-6)
loss_function = nn.NLLLoss()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [0]:
def train_lstm(train_dataset, labels, model, optimizer, batch_size):
   
    train_predictions = np.array([])
    pred_labels = np.array([])
    model.train()
    train_loss = 0
 #   out_labels = []
    
    for texts, labs in iterate_minibatches(np.array(train_dataset), np.array(labels),\
                                           batch_size=batch_size, shuffle=True, verbose=True):
        #emb_ix = generate_batch(embeddings[i: i + batch_size])
        emb_ix, lengths = generate_embeddings(texts,  src_embeddings, src_word2id, maxLen=None)
        emb_ix = torch.tensor(emb_ix).float().to(DEVICE)
        lengths = torch.tensor(lengths).to(DEVICE)
    #    print(emb_ix.type())
    #    print(lengths.type())
        emb_ix.requires_grad_(False)
        res_labels = torch.tensor(labs).to(DEVICE)
        optimizer.zero_grad()
        output = model(emb_ix, lengths)             #lengths
        loss = loss_function(output, res_labels)
        loss.backward()
     #   torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), 
      #                                     max_norm=1.)
        optimizer.step()

        #print(loss)
        train_loss += loss.item()
        
        y_pred = output.argmax(1).cpu().numpy()
        train_predictions = np.hstack((train_predictions, y_pred))
        pred_labels = np.hstack((pred_labels, res_labels.cpu().numpy()))

    train_f1_micro = f1_score(pred_labels, train_predictions, average='micro')*100
    train_f1_weighted = f1_score(pred_labels, train_predictions, average='weighted')*100

#    torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict(),
 #               'loss': loss
  #          }, '/content/gdrive/My Drive/cross embeddings experiments/BiLSTM_abs_model_checkpoint')

    return train_loss / ((len(train_dataset)//batch_size)+1), train_f1_micro, train_f1_weighted

In [0]:
def test_lstm(test_dataset, labels, model, batch_size, mode):
    model.eval()
    test_loss = 0
    test_predictions = np.array([])
    
    for texts, labs in iterate_minibatches(np.array(test_dataset), np.array(labels),\
                                           batch_size=batch_size, shuffle=False, verbose=True):
        if mode == 'val':
            emb_ix, lengths = generate_embeddings(texts,  src_embeddings, src_word2id, maxLen=None)
        else:
            emb_ix, lengths = generate_embeddings(texts,  tgt_embeddings, tgt_word2id, maxLen=None)
        emb_ix = torch.tensor(emb_ix).float().to(DEVICE)
        lengths = torch.tensor(lengths).to(DEVICE)
        res_labels = torch.tensor(labs).to(DEVICE) 
        output = model(emb_ix, lengths)          #lengths
    
        y_pred = output.argmax(1).cpu().numpy()
        test_predictions = np.hstack((test_predictions, y_pred))

    test_f1_micro = f1_score(labels, test_predictions, average='micro')*100
    test_f1_weighted = f1_score(labels, test_predictions, average='weighted')*100

    return test_f1_micro, test_f1_weighted

In [0]:
def iterate_minibatches(data, labels, batch_size=100, shuffle=True, verbose=True):
    indices = np.arange(len(data))
    if shuffle:
        indices = np.random.permutation(indices)        
    irange = tnrange if verbose else range
    for start in irange(0, len(indices), batch_size):
        yield data[indices[start: start + batch_size]], labels[indices[start: start + batch_size]]

In [0]:
#for random search
from itertools import product
num_layers = range(1, 3)
hidden_dims = range(128, 512, 50)
dropouts = np.arange(0.0, 0.2, 0.1)
params = list(product(hidden_dims, num_layers, dropouts))
np.random.shuffle(params)

In [0]:
params[0]

(278, 1, 0.0)

In [0]:
def training(model, hidden_size, layers, dropout=0):
    print('Training...')
    f1_best_micro = 0
    for epoch in tnrange(N_EPOCHS+1):
        print('Epoch #', epoch)
        train_loss, train_f1_micro, train_f1_weighted = train_lstm(train_texts, y_train, model, optimizer, batch_size=64)
        val_f1_micro, val_f1_weighted = test_lstm(val_texts, y_val, model, batch_size=64, mode='val')
        if val_f1_micro < f1_best_micro:
            break
        else:
            f1_best_micro = val_f1_micro
            weights = copy.deepcopy(model.state_dict())
            new_model = LSTMClassifier(hidden_size=hidden_size, layers=layers, dropout = dropout).to(DEVICE)
            new_model.load_state_dict(weights)
        print(f'\nTrain loss: {train_loss:.8f} | Train f1 micro: {train_f1_micro:.4f}  | Train f1 weighted: {train_f1_weighted:.4f}' )   
        #   print(f'Test accuracy {test_acc:.4f}')
        print(f'\nVal f1 micro: {val_f1_micro:.4f} | Val f1 weighted: {val_f1_weighted:.4f}')
    print('Finished training.')
    print('Best f1-micro: ', f1_best_micro)
    test_f1_micro, test_f1_weighted = test_lstm(test_texts, y_test, new_model, batch_size=64, mode='test')
    print('Test f1-micro: ', test_f1_micro)
    print('Test f1-weighted: ', test_f1_weighted)
  #  return new_model

In [0]:
for param in params[:5]:
    print('hidden_size= ', param[0])
    print('num_layers= ', param[1])
    print('dropout= ', param[2])
    N_EPOCHS = 15
    LEARNING_RATE = 1e-3
    model = LSTMClassifier(hidden_size=param[0], layers=param[1], dropout=param[2]).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
    loss_function = nn.NLLLoss()
    training(model, param[0], param[1], param[2])

hidden_size=  278
num_layers=  1
dropout=  0.0
Training...


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  after removing the cwd from sys.path.


Epoch # 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  




Train loss: 4.81768837 | Train f1 micro: 3.4402  | Train f1 weighted: 2.3670

Val f1 micro: 9.0587 | Val f1 weighted: 4.6117
Epoch # 1


...



Train loss: 2.38219405 | Train f1 micro: 39.9884  | Train f1 weighted: 38.2673

Val f1 micro: 43.9265 | Val f1 weighted: 41.8337
Epoch # 3




Train loss: 2.12031988 | Train f1 micro: 46.1638  | Train f1 weighted: 44.8957

Val f1 micro: 46.6277 | Val f1 weighted: 45.4237
Epoch # 4




Train loss: 1.94676970 | Train f1 micro: 50.3408  | Train f1 weighted: 49.3043

Val f1 micro: 49.2959 | Val f1 weighted: 47.7909
Epoch # 5



Finished training.
Best f1-micro:  49.295890636580744



Test f1-micro:  24.52216748768473
Test f1-weighted:  21.99681779554012


In [0]:
test_f1_micro, test_f1_weighted = test_lstm(test_texts, y_test, model, batch_size=64, mode='test')
test_f1_micro

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  


24.374384236453203

In [0]:
#torch.save(new_model.state_dict(), '/content/gdrive/My Drive/cross embeddings experiments/BiLSTM_abs_bestmodel')

#### CNN

In [0]:
def iterate_minibatches(data, labels, batch_size=100, shuffle=True, verbose=True):
    indices = np.arange(len(data))
    if shuffle:
        indices = np.random.permutation(indices)        
    irange = tnrange if verbose else range
    for start in irange(0, len(indices), batch_size):
        yield data[indices[start: start + batch_size]], labels[indices[start: start + batch_size]]

In [0]:
class CNN(nn.Module):
    def __init__(self, kernel_sizes, output_size=300, dropout=0.5):
        super(CNN, self).__init__()
        self.convs = nn.ModuleList([nn.Conv1d(300, output_size, kernel_size=K) for K in kernel_sizes])
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.linear = nn.Linear(output_size*3, 225)
        self.dropout = dropout
      #  self.linear2 = nn.Linear(256, 2)
        
    def forward(self, x):
        
        x = x.permute(0,2,1)
        x = [torch.relu(conv(x)) for conv in self.convs]
        x = [self.pool(elem) for elem in x]
        x = torch.cat(x, dim=1).squeeze(2)
    #    out = self.linear2(torch.relu((self.linear(nn.Dropout(dropout)(x)))))
        out = self.linear(nn.Dropout(self.dropout)(x))
        return F.log_softmax(out, dim=1)

In [0]:
def train_cnn(train_dataset, labels, model, optimizer, batch_size):
   
    train_predictions = np.array([])
    pred_labels = np.array([])
    model.train()
    train_loss = 0
 #   out_labels = []
    
    for texts, labs in iterate_minibatches(np.array(train_dataset), np.array(labels),\
                                           batch_size=batch_size, shuffle=True, verbose=True):
        emb_ix, _ = generate_embeddings(texts,  src_embeddings, src_word2id, maxLen=None)
        emb_ix = torch.tensor(emb_ix).float().to(DEVICE)
        emb_ix.requires_grad_(False)
        res_labels = torch.tensor(labs).to(DEVICE)
        optimizer.zero_grad()
        output = model(emb_ix)
        loss = loss_function(output, res_labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
        y_pred = output.argmax(1).cpu().numpy()
        train_predictions = np.hstack((train_predictions, y_pred))
        pred_labels = np.hstack((pred_labels, res_labels.cpu().numpy()))

    train_f1_micro = f1_score(pred_labels, train_predictions, average='micro')*100
    train_f1_weighted = f1_score(pred_labels, train_predictions, average='weighted')*100

#    torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict(),
 #               'loss': loss
  #          }, '/content/gdrive/My Drive/cross embeddings experiments/BiLSTM_abs_model_checkpoint')

    return train_loss / ((len(train_dataset)//batch_size)+1), train_f1_micro, train_f1_weighted

In [0]:
def test_cnn(test_dataset, labels, model, batch_size, mode='val'):
    model.eval()
    test_loss = 0
    test_predictions = np.array([])
    
    for texts, labs in iterate_minibatches(np.array(test_dataset), np.array(labels),\
                                           batch_size=batch_size, shuffle=False, verbose=True):
        if mode == 'val':
            emb_ix, _ = generate_embeddings(texts,  src_embeddings, src_word2id, maxLen=None)
        else:
            emb_ix, _ = generate_embeddings(texts,  tgt_embeddings, tgt_word2id, maxLen=None)
        emb_ix = torch.tensor(emb_ix).float().to(DEVICE)
        res_labels = torch.tensor(labs).to(DEVICE) 
        output = model(emb_ix)
    
        y_pred = output.argmax(1).cpu().numpy()
        test_predictions = np.hstack((test_predictions, y_pred))

    test_f1_micro = f1_score(labels, test_predictions, average='micro')*100
    test_f1_weighted = f1_score(labels, test_predictions, average='weighted')*100

    return test_f1_micro, test_f1_weighted

In [0]:
N_EPOCHS = 10
LEARNING_RATE = 1e-3
model = CNN(kernel_sizes=[3,4,5]).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
loss_function = nn.NLLLoss()

In [0]:
#kernel_sizes = [3,4,5]
print('Training...')
f1_best_micro = 0
for epoch in tnrange(N_EPOCHS+1):
    print('Epoch #', epoch)
    train_loss, train_f1_micro, train_f1_macro = train_cnn(train_texts, y_train, model, optimizer, batch_size=64)
    test_f1_micro, test_f1_macro = test_cnn(val_texts, y_val, model, batch_size=64)
    if test_f1_micro < f1_best_micro:
        break
    else:
        f1_best_micro = test_f1_micro
        weights = copy.deepcopy(model.state_dict())
        new_model = CNN(kernel_sizes=[3,4,5]).to(DEVICE)
        new_model.load_state_dict(weights)
    print(f'\nTrain loss: {train_loss:.8f} | Train f1 micro: {train_f1_micro:.4f}  | Train f1 weighted: {train_f1_macro:.4f}' )   
    #   print(f'Test accuracy {test_acc:.4f}')
    print(f'\nTest f1 micro: {test_f1_micro:.4f} | Test f1 weighted: {test_f1_macro:.4f}')
print('Finished training.')
print('Best f1-micro: ', f1_best_micro)

Training...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch # 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  




Train loss: 3.81976877 | Train f1 micro: 18.2792  | Train f1 weighted: 15.9864

Test f1 micro: 33.7561 | Test f1 weighted: 30.2878
Epoch # 1


...



Train loss: 2.16827711 | Train f1 micro: 46.8992  | Train f1 weighted: 45.2749

Test f1 micro: 45.9524 | Test f1 weighted: 44.4200
Epoch # 3




Train loss: 2.00249275 | Train f1 micro: 50.3713  | Train f1 weighted: 49.1370

Test f1 micro: 47.4594 | Test f1 weighted: 46.0190
Epoch # 4




Train loss: 1.88728943 | Train f1 micro: 52.7142  | Train f1 weighted: 51.7272

Test f1 micro: 49.0900 | Test f1 weighted: 47.8947
Epoch # 5




Train loss: 1.79480189 | Train f1 micro: 54.8507  | Train f1 weighted: 53.9060

Test f1 micro: 49.8641 | Test f1 weighted: 49.0206
Epoch # 6



Finished training.
Best f1-micro:  49.864119245655935


In [0]:
test_cnn(test_texts, y_test, model, batch_size=64, mode='test')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  


(23.48768472906404, 21.794173399586978)

In [0]:
def training_cnn(model, kernel_sizes, output_size, dropout):
    print('Training...')
    f1_best_micro = 0
    for epoch in tnrange(N_EPOCHS+1):
        print('Epoch #', epoch)
        train_loss, train_f1_micro, train_f1_weighted = train_cnn(train_texts, y_train, model, optimizer, batch_size=64)
        val_f1_micro, val_f1_weighted = test_cnn(val_texts, y_val, model, batch_size=64, mode='val')
        if val_f1_micro < f1_best_micro:
            break
        else:
            f1_best_micro = val_f1_micro
            weights = copy.deepcopy(model.state_dict())
            new_model = CNN(kernel_sizes=kernel_sizes, output_size=output_size, dropout=dropout).to(DEVICE)
            new_model.load_state_dict(weights)
        print(f'\nTrain loss: {train_loss:.8f} | Train f1 micro: {train_f1_micro:.4f}  | Train f1 weighted: {train_f1_weighted:.4f}' )   
        #   print(f'Test accuracy {test_acc:.4f}')
        print(f'\nVal f1 micro: {val_f1_micro:.4f} | Val f1 weighted: {val_f1_weighted:.4f}')
    print('Finished training.')
    print('Best f1-micro: ', f1_best_micro)
    test_f1_micro, test_f1_weighted = test_cnn(test_texts, y_test, new_model, batch_size=64, mode='test')
    print('Test f1-micro: ', test_f1_micro)
    print('Test f1-weighted: ', test_f1_weighted)
  #  return new_model

In [0]:
#for random search
from itertools import product
kernel_sizes = [[2,3,4], [3,4,5]]
output_size = range(128, 512, 50)
dropouts = np.arange(0.4, 0.8, 0.1)
params = list(product(kernel_sizes, output_size, dropouts))
np.random.shuffle(params)

In [0]:
for param in params[:5]:
    print('kernel_sizes= ', param[0])
    print('output_size= ', param[1])
    print('dropout= ', param[2])
    N_EPOCHS = 15
    LEARNING_RATE = 1e-3
    model = CNN(kernel_sizes=param[0], output_size=param[1], dropout=param[2]).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
    loss_function = nn.NLLLoss()
    training_cnn(model, param[0], param[1], param[2])

kernel_sizes=  [3, 4, 5]
output_size=  128
dropout=  0.6
Training...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  after removing the cwd from sys.path.


Epoch # 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  




Train loss: 4.32519924 | Train f1 micro: 10.4585  | Train f1 weighted: 8.2902

Val f1 micro: 21.6586 | Val f1 weighted: 17.4776
Epoch # 1


...



Train loss: 1.97424194 | Train f1 micro: 50.9643  | Train f1 weighted: 49.9779

Val f1 micro: 46.5289 | Val f1 weighted: 45.4818
Epoch # 8




Train loss: 1.93255204 | Train f1 micro: 52.0456  | Train f1 weighted: 51.1476

Val f1 micro: 46.8583 | Val f1 weighted: 45.6707
Epoch # 9




Train loss: 1.89414433 | Train f1 micro: 52.6095  | Train f1 weighted: 51.7721

Val f1 micro: 47.4183 | Val f1 weighted: 46.2326
Epoch # 10



Finished training.
Best f1-micro:  47.418265667462734



Test f1-micro:  21.093596059113302
Test f1-weighted:  18.83178863338573


In [0]:
for param in params[5:10]:
    print('kernel_sizes= ', param[0])
    print('output_size= ', param[1])
    print('dropout= ', param[2])
    N_EPOCHS = 15
    LEARNING_RATE = 1e-3
    model = CNN(kernel_sizes=param[0], output_size=param[1], dropout=param[2]).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
    loss_function = nn.NLLLoss()
    training_cnn(model, param[0], param[1], param[2])

kernel_sizes=  [2, 3, 4]
output_size=  228
dropout=  0.7
Training...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  after removing the cwd from sys.path.


Epoch # 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  




Train loss: 4.21571188 | Train f1 micro: 12.6444  | Train f1 weighted: 10.5405

Val f1 micro: 24.4668 | Val f1 weighted: 20.7424
Epoch # 1


...



Train loss: 2.02789867 | Train f1 micro: 49.8917  | Train f1 weighted: 48.7370

Val f1 micro: 46.3724 | Val f1 weighted: 45.0325
Epoch # 6




Train loss: 1.96048158 | Train f1 micro: 51.4570  | Train f1 weighted: 50.4210

Val f1 micro: 47.1877 | Val f1 weighted: 46.0001
Epoch # 7




Train loss: 1.90507479 | Train f1 micro: 52.5035  | Train f1 weighted: 51.5888

Val f1 micro: 47.7312 | Val f1 weighted: 46.4364
Epoch # 8



Finished training.
Best f1-micro:  47.73120316231574



Test f1-micro:  22.354679802955665
Test f1-weighted:  20.68820304003764
